In [1]:
import sys
import os

# For Jupyter notebooks
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Import commonly used modules
from data_utils import *
from model_utils import *
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
from numba import cuda
import gc
from tensorflow.keras import backend as K


from config_utils import load_config

# Load default config
CONFIG = load_config('../config/fl_template_config.yaml')

# Export commonly used items
__all__ = ['CONFIG']

2025-03-25 13:32:54.172018: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742931174.189427    5842 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742931174.194730    5842 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742931174.213034    5842 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742931174.213064    5842 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742931174.213066    5842 computation_placer.cc:177] computation placer alr

## FedAvg

### Federated Learning Averaging Pseudocode

### Server Initialization:
Initialize global model weights W₀

### Main Federated Learning Loop:
For each round t = 1 to T:
    
    1. Select a subset of clients Sₜ (or use all available clients)
    
    2. Broadcast the current global model weights Wₜ to all clients in Sₜ

    3. For each client k in Sₜ (executed in parallel):
         - Perform a local update:
           Wₜᵏ = ClientUpdate(Wₜ, local_dataₖ)
         - Return updated local model weights Wₜᵏ along with the number of samples nₖ

    4. Aggregate the updated weights using weighted averaging (FedAvg):
         - Compute total samples: N = Σₖ₍∈Sₜ₎ nₖ
         - Update global model weights:
           Wₜ₊₁ = Σₖ₍∈Sₜ₎ (nₖ / N) * Wₜᵏ

Return final global model weights W_T

### ClientUpdate Function:
Function ClientUpdate(W, local_data):
    
    - Set W_local = W
    
    - For each local epoch e = 1 to E:
         - For each batch b in local_data:
              - Compute gradient: grad = ∇(loss(W_local, b))
              - Update local weights: W_local = W_local - learning_rate * grad
              
    Return W_local

#### Central Model Initialization

In [2]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [3]:
history_dic = {}

In [4]:
model = create_model()
trainx,trainy = load_training_data(f'../experiments/{CONFIG['experiment_name']}/processed_data/init.npy')
history =  train_model(model,trainx,trainy)
history_dic['init'] = history
save_model(model,f'../experiments/{CONFIG['experiment_name']}/models/central_model.keras')
del model
K.clear_session()
gc.collect()

/home/yash/miniconda3/envs/fedml/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1742931179.246352    5842 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5582 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Epoch 1/10


I0000 00:00:1742931181.881267    5923 service.cc:152] XLA service 0x7fc4d000f360 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1742931181.881316    5923 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Laptop GPU, Compute Capability 8.6
2025-03-25 13:33:01.937529: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1742931182.261379    5923 cuda_dnn.cc:529] Loaded cuDNN version 90300


1/7 ━━━━━━━━━━━━━━━━━━━━ 31s 5s/step - accuracy: 0.7500 - auc: 0.6636 - loss: 0.6730 - precision: 0.6667 - recall: 0.4000

I0000 00:00:1742931185.441751    5923 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 677ms/step - accuracy: 0.6235 - auc: 0.5452 - loss: 1.1168 - precision: 0.6877 - recall: 0.5327
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6074 - auc: 0.8511 - loss: 0.5936 - precision: 0.5811 - recall: 0.9803
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9183 - auc: 0.9574 - loss: 0.3374 - precision: 0.9136 - recall: 0.9321
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9183 - auc: 0.9687 - loss: 0.1966 - precision: 0.9478 - recall: 0.8816
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8842 - auc: 0.9372 - loss: 0.3660 - precision: 0.9107 - recall: 0.8725
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9032 - auc: 0.9678 - loss: 0.3291 - precision: 0.8662 - recall: 0.9607
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9322 - auc: 0.9676 - loss: 0.2258 - precision: 1.0000 - recall: 0.8670
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9560 - au

0

#### Client Models Initialization

In [5]:
central_model = load_model_from_disk(f'../experiments/{CONFIG['experiment_name']}/models/central_model.keras')
for i in range(CONFIG.get('num_clients',5)):
    model = create_model(seed = i+1)
    model.set_weights(central_model.get_weights())
    model._name = f'client_model_{i+1}'
    save_model(model,f'../experiments/{CONFIG['experiment_name']}/models/{model._name}.keras')
    del model
    K.clear_session()
    gc.collect()

📦 Model loaded from: ../experiments/fed_ml_experiment_1/models/central_model.keras
💾 Model saved to: ../experiments/fed_ml_experiment_1/models/client_model_1.keras
💾 Model saved to: ../experiments/fed_ml_experiment_1/models/client_model_2.keras
💾 Model saved to: ../experiments/fed_ml_experiment_1/models/client_model_3.keras
💾 Model saved to: ../experiments/fed_ml_experiment_1/models/client_model_4.keras
💾 Model saved to: ../experiments/fed_ml_experiment_1/models/client_model_5.keras


#### Local Train Client models and Global Aggregation

In [ ]:
delta = 0

## FedAvg Conditioned

### Modified FedAvg with Performance-based Weighting

### Server Initialization:
Initialize global model weights W₀

### Main Federated Learning Loop:

For each round t = 1 to T:

    1. Select a subset of clients Sₜ (or use all available clients)
    2. Broadcast the current global model weights Wₜ to all clients in Sₜ

    3. For each client k in Sₜ (executed in parallel):
         - Perform a local update:
           Wₜᵏ = ClientUpdate(Wₜ, local_dataₖ)
         - Evaluate the updated model on a common validation set:
           aₖ = Evaluate(Wₜᵏ, validation_set)  # e.g., accuracy
         - Return updated model Wₜᵏ, number of samples nₖ, and accuracy aₖ

    4. Aggregate the updated weights:
         - Compute the performance-weighted sum of samples:
           Total_weight = Σₖ₍∈Sₜ₎ (nₖ × aₖ)
         - Update global model weights:
           Wₜ₊₁ = Σₖ₍∈Sₜ₎ [(nₖ × aₖ) / Total_weight] × Wₜᵏ

Return final global model weights W_T

### ClientUpdate Function:

Function ClientUpdate(W, local_data):
    
    Set W_local = W
    For each local epoch e = 1 to E:
         For each batch b in local_data:
              - Compute gradient: grad = ∇(loss(W_local, b))
              - Update local weights: W_local = W_local - learning_rate * grad
    Return W_local


## Asynchronous Weight Updating Federated Learning

### Federated Learning with Partial Weight Sharing (Deep Layers Updated Frequently)

### Server Initialization:
Initialize global shallow weights W_shallow₀
Initialize global deep weights W_deep₀
Set shallow_update_interval K  # e.g., update shallow layers every K rounds, update deep layers every round

### Main Federated Learning Loop:

For each round t = 1 to T:

    1. Determine if this is a shallow update round:
         If (t mod K == 0):
             shallow_update = True
         Else:
             shallow_update = False

    2. Client Selection & Broadcast:
         Select a subset of clients Sₜ
         For each client in Sₜ, send:
             - Current deep weights: W_deepₜ  (always sent)
             - If shallow_update is True, also send current shallow weights: W_shallowₜ
             - Otherwise, clients use their locally stored shallow weights

    3. Clients' Local Update (executed in parallel):
         For each client k in Sₜ:
             - If shallow_update is True:
                  (W_shallowₜ^k, W_deepₜ^k) = ClientUpdate(W_shallowₜ, W_deepₜ, local_dataₖ, update_shallow=True)
             - Else:
                  (W_shallow_local, W_deepₜ^k) = ClientUpdate(W_shallow_local, W_deepₜ, local_dataₖ, update_shallow=False)
             - Evaluate the full updated model on a common validation set:
                  aₖ = Evaluate(FullModel(W_shallow, W_deep), validation_set)  # e.g., accuracy
             - Return to server:
                  - For shallow layers: if update_shallow is True, return updated W_shallowₜ^k; otherwise, no update (or the previous version)
                  - Updated deep weights: W_deepₜ^k
                  - Local sample count nₖ and performance metric aₖ

    4. Server Aggregation:
         # Always aggregate deep layers:
         Compute Total_weight_deep = Σₖ₍∈Sₜ₎ (nₖ × aₖ)
         Update global deep weights:
             W_deepₜ₊₁ = Σₖ₍∈Sₜ₎ [ (nₖ × aₖ) / Total_weight_deep ] × W_deepₜ^k

         # Aggregate shallow layers only on shallow update rounds:
         If shallow_update is True:
             Compute Total_weight_shallow = Σₖ₍∈Sₜ₎ (nₖ × aₖ)
             Update global shallow weights:
                 W_shallowₜ₊₁ = Σₖ₍∈Sₜ₎ [ (nₖ × aₖ) / Total_weight_shallow ] × W_shallowₜ^k
         Else:
             W_shallowₜ₊₁ = W_shallowₜ  # Keep shallow layers unchanged

    Return final global model: {W_shallow_T, W_deep_T}


### ClientUpdate Function:

Function ClientUpdate(shallow_weights, deep_weights, local_data, update_shallow):

    If update_shallow is True:
         Set local_shallow = shallow_weights    # Received from server
    Else:
         Set local_shallow = local_shallow      # Use previously stored shallow weights locally

    Set local_deep = deep_weights              # Always use the latest deep weights from server

    For each local epoch e = 1 to E:
         For each batch b in local_data:
              If update_shallow is True:
                  - Compute gradients for both layers:
                        grad_shallow, grad_deep = ∇(loss(FullModel(local_shallow, local_deep), b))
                  - Update shallow layers:
                        local_shallow = local_shallow - learning_rate * grad_shallow
              Else:
                  - Compute gradient only for deep layers (shallow remains fixed):
                        grad_deep = ∇(loss(FullModel(local_shallow, local_deep), b))
              - Update deep layers:
                    local_deep = local_deep - learning_rate * grad_deep

    If update_shallow is True:
         Return (local_shallow, local_deep)
    Else:
         Return (local_shallow, local_deep)  # Note: shallow remains unchanged from before the round
